In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
import math as m

In [8]:
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = [
        'sepal length', 'sepal width', 'petal length', 'petal width', 'label'
    ]
    data = np.array(df.iloc[:100, :])
    return data[:, :-1], data[:, -1]

In [12]:
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### GaussianNB model

In [73]:
class NaiveBayes:
    
    def __init__(self):
        self.model = None # Store the label and the mean / stdev of each feature
        
    def mean(self, X):
        return sum(X) / float(len(X))
    
    def stdev(self, X):
        avg = self.mean(X)
        return m.sqrt(sum([m.pow(x - avg, 2) for x in X]) / float(len(X)))
    
    def gaussain_pdf(self, x, mean, stdev):
        exp = m.exp(-m.pow(x - mean, 2) / (2 * m.pow(stdev, 2)))
        return exp / (m.sqrt(2 * m.pi) * stdev)

    def summarize(self, train_data):
        return [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]
    
    def fit(self, X, y):
        labels = list(set(y))
        data = {label: [] for label in labels}
        for f, label in zip(X, y):
            data[label].append(f)
        self.model = {
            label: self.summarize(value)
            for label, value in data.items()
        }       
        return 'Gaussian Training Done!'
    
    def predict(self, X_test):
        probabilities = {}
        for label, value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)):
                mean, stdev = value[i]
                probabilities[label] *= self.gaussain_pdf(X_test[i], mean, stdev)
        label = sorted(
            probabilities.items(),
            key=lambda x:x[1]
        )[-1][0]
        return label
    
    def score(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            if self.predict(X) == y:
                right += 1
        return right / float(len(y_test))    

In [74]:
model = NaiveBayes()
model.fit(X_train, y_train)

'Gaussian Training Done!'

In [87]:
model.predict([1, 3, 5, 4])

1.0